## Check for structure and format errors

Kingfisher Process checks data against the OCDS schema using [CoVE](https://github.com/OpenDataServices/cove). For release collections, Kingfisher Process stores check results in the `release_check` table. For record collections, Kingfisher Process stores check results in the `record_check` table.

### Confirm that checks are complete

By default, Kingfisher Process checks all data, therefore there is often a long queue of collections to be checked. Use the following query to confirm that checks are complete for your collection(s).

If checks for your collection(s) have not started yet, you can use the [`check collection` command](https://kingfisher-process.readthedocs.io/en/latest/cli/check-collection.html) to start the checks manually.

If checks are in progress, you should wait for the checks to finish before running the queries in this section.

In [ ]:
%%sql

SELECT
    collection_id,
    'release' AS collection_type,
    CASE WHEN count(release.id) = count(release_check.id) THEN
        'complete'
    WHEN count(release_check.id) = 0 THEN
        'not_started'
    ELSE
        'in_progress'
    END AS check_status,
    count(release_check.id)::text || '/' || count(release.id)::text AS check_progress
FROM
    release_check
    RIGHT JOIN release ON release_check.release_id = release.id
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id
UNION
SELECT
    collection_id,
    'record' AS collection_type,
    CASE WHEN count(record.id) = count(record_check.id) THEN
        'complete'
    WHEN count(record_check.id) = 0 THEN
        'not_started'
    ELSE
        'in_progress'
    END AS check_status,
    count(record_check.id)::text || '/' || count(record.id)::text AS check_progress
FROM
    record_check
    RIGHT JOIN record ON record_check.record_id = record.id
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id;


### Error summary

Generate a summary of errors from the `release_check` and `record_check` tables.

In [ ]:
%%sql structure_and_format_error_summary <<

WITH errors AS (
    SELECT
        collection_id,
        errors ->> 'type' AS error_type,
    LEFT (errors ->> 'description',
        49000) AS error,
    ocid,
    errors ->> 'value' AS value,
    row_number() OVER (PARTITION BY collection_id,
        errors ->> 'type',
    LEFT (errors ->> 'description',
    49000)) AS rownum
FROM
    release_check rc
    CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    JOIN RELEASE r ON rc.release_id = r.id
WHERE
    collection_id IN :collection_ids
UNION ALL
SELECT
    collection_id,
    errors ->> 'type' AS error_type,
    LEFT (errors ->> 'description',
        49000) AS error,
    ocid,
    errors ->> 'value' AS value,
    row_number() OVER (PARTITION BY collection_id,
        errors ->> 'type',
    LEFT (errors ->> 'description',
    49000)) AS rownum
FROM
    record_check rc
    CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    JOIN record r ON rc.record_id = r.id
WHERE
    collection_id IN :collection_ids
),
examples AS (
    SELECT
        collection_id,
        error_type,
        error,
        array_agg(ocid) AS example_ocids,
        array_agg(value) AS example_values
    FROM
        errors
    WHERE
        rownum <= 3
    GROUP BY
        collection_id,
        error_type,
        error
)
SELECT
    collection_id,
    error_type,
    error,
    count(*) AS count,
    example_ocids,
    example_values
FROM
    errors
    JOIN examples USING (collection_id, error_type, error)
GROUP BY
    collection_id,
    error_type,
    error,
    example_ocids,
    example_values;



In [ ]:
structure_and_format_error_summary

### Error details

Generate a full list of errors from the `release_check` and `record_check` tables.

In [ ]:
%%sql structure_and_format_errors <<

SELECT
    collection_id,
    'release' AS collection_type,
    errors ->> 'type' AS error_type,
    errors ->> 'field' AS field,
    LEFT (errors ->> 'description',
        49000) AS error,
    ocid,
    errors ->> 'value' AS value
FROM
    release_check rc
    CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    JOIN RELEASE r ON rc.release_id = r.id
WHERE
    collection_id IN :collection_ids
UNION ALL
SELECT
    collection_id,
    'record' AS collection_type,
    errors ->> 'type' AS error_type,
    errors ->> 'field' AS field,
    LEFT (errors ->> 'description',
        49000) AS error,
    ocid,
    errors ->> 'value' AS value
FROM
    record_check rc
    CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    JOIN record r ON rc.record_id = r.id
WHERE
    collection_id IN :collection_ids


In [ ]:
structure_and_format_errors